In [1]:
from OurTrainingTools2D import *

=========== Random Seed: 96 ===========


In [2]:
class OurCDModel(nn.Module):
### Defines the  model with parametrized discriminant. Only quadratic dependence on a single parameter is implemented.
### Input is the architecture (list of integers, the last one being equal to 1) and the activation type ('ReLU' or 'Sigmoid')
    def __init__(self, NumberOfParameters, AR = [1, 3, 3, 1] , AF = 'ReLU'):               
        super(OurCDModel, self).__init__() 
        ValidActivationFunctions = {'ReLU': torch.relu, 'Sigmoid': torch.sigmoid}
        try:
            self.ActivationFunction = ValidActivationFunctions[AF]
        except KeyError:
            print('The activation function specified is not valid. Allowed activations are %s.'
                 %str(list(ValidActivationFunctions.keys())))
            print('Will use ReLU.')
            self.ActivationFunction = torch.relu            
        if type(AR) == list:
            if( ( all(isinstance(n, int) for n in AR)) and ( AR[-1] == 1) ):
                self.Architecture = AR
            else:
                print('Architecture should be a list of integers, the last one should be 1.')
                raise ValueError             
        else:
            print('Architecture should be a list !')
            raise ValueError
        self.DefineLayers(NumberOfParameters)

### Define Layers
    def DefineLayers(self, NumberOfParameters):
        print('====== Defining layers for %d parameters. ======'%(NumberOfParameters))
        self.NumberOfParameters = NumberOfParameters
        self.NumberOfNetworks = int((2+self.NumberOfParameters)*(1+self.NumberOfParameters)/2)-1
        LinearLayers = [([nn.Linear(self.Architecture[i], self.Architecture[i+1]) \
                                  for i in range(len(self.Architecture)-1)])\
                        for n in range(self.NumberOfNetworks)]
        LinearLayers = [Layer for SubLayerList in LinearLayers for Layer in SubLayerList]
        self.LinearLayers = nn.ModuleList(LinearLayers)
    
    def Forward(self, Data, Parameters):
### Forward Function. Performs Preprocessing, returns F = rho/(1+rho) in [0,1], where rho is quadratically parametrized.
        # Checking that data has the right input dimension
        InputDimension = self.Architecture[0]
        if Data.size(1) != InputDimension:
            print('Dimensions of the data and the network input mismatch: data: %d, model: %d'
                  %(Data.size(1), InputDimension))
            raise ValueError

        # Checking that preprocess has been initialised
        if not hasattr(self, 'Shift'):
            print('Please initialize preprocess parameters!')
            raise ValueError
        if not hasattr(self, 'IsParamRedundant'):
            print('Please make sure that you have checked for Parameter redundancy.')
            raise ValueError
            
        if self.IsParamRedundant:
            Parameters = self.Parameters_Short
            
        with torch.no_grad(): 
            Data, Parameters = self.Preprocess(Data, Parameters)  
        
        print(Parameters)
        
        NumberOfLayers, NumberOfEvents = len(self.Architecture)-1, Data.size(0)
        EntryIterator, NetworkIterator = 0, -1
        MatrixLT = torch.zeros([NumberOfEvents, (self.NumberOfParameters+1)**2], dtype=Data.dtype)
        
        if Data.is_cuda:
            MatrixLT = OurCudaTensor(MatrixLT)
        
        for i in range(self.NumberOfParameters+1):
            EntryIterator += i
            DiagonalEntry = True
            for j in range(self.NumberOfParameters+1-i):
                if NetworkIterator == -1:
                    MatrixLT[:, EntryIterator] = torch.ones(NumberOfEvents)
                    #print('Entry: %d, Layer: ones, DiagonalEntry: %s'%(EntryIterator,
                    #                                                str(DiagonalEntry)))
                else:
                    x = Data
                    for Layer in self.LinearLayers[NumberOfLayers*NetworkIterator:\
                                                  NumberOfLayers*(NetworkIterator+1)-1]:
                        x = self.ActivationFunction(Layer(x))
                    x = self.LinearLayers[NumberOfLayers*(NetworkIterator+1)-1](x).squeeze()
                    #MatrixLT[:, EntryIterator] = torch.exp(x) if DiagonalEntry else x
                    MatrixLT[:, EntryIterator] = x
                    #print('Entry: %d, Layer: %d, DiagonalEntry: %s'%(EntryIterator, NetworkIterator, 
                    #                                                str(DiagonalEntry)))
                EntryIterator += 1
                NetworkIterator += 1
                DiagonalEntry = False
        #print('MatrixLT: '+str(MatrixLT.is_cuda))
        #print('Parameters: '+str(Parameters.is_cuda))

        MatrixLT = MatrixLT.reshape([-1, self.NumberOfParameters+1, self.NumberOfParameters+1]) 
        MatrixLTP = MatrixLT.matmul(Parameters.reshape([NumberOfEvents, self.NumberOfParameters+1, 1]))
        rho = MatrixLTP.permute([0, 2, 1]).matmul(MatrixLTP).squeeze()
        
        return (rho.div(1.+rho)).view(-1, 1)
    
    def GetL1Bound(self, L1perUnit):
        self.L1perUnit = L1perUnit
    
    def ClipL1Norm(self):
### Clip the weights      
        def ClipL1NormLayer(DesignatedL1Max, Layer, Counter):
            if Counter == 1:
                ### this avoids clipping the first layer
                return
            L1 = Layer.weight.abs().sum()
            Layer.weight.masked_scatter_(L1 > DesignatedL1Max, 
                                        Layer.weight*(DesignatedL1Max/L1))
            return
        
        Counter = 0
        for m in self.children():
            if isinstance(m, nn.Linear):
                Counter += 1
                with torch.no_grad():
                    DesignatedL1Max = m.weight.size(0)*m.weight.size(1)*self.L1perUnit
                    ClipL1NormLayer(DesignatedL1Max, m, Counter)
            else:
                for mm in m:
                    Counter +=1
                    with torch.no_grad():
                        DesignatedL1Max = mm.weight.size(0)*m.weight.size(1)*self.L1perUnit
                        ClipL1NormLayer(DesignatedL1Max, mm, Counter)
        return 
    
    def DistributionRatio(self, points):
### This is rho. I.e., after training, the estimator of the distribution ratio.
        with torch.no_grad():
            F = self(points)
        return F/(1-F)
    
    def checkRedundancy(self, Parameters):
### This is written specifically for 2D networks. It will check if any columns of the parameters are redundant 
### (i.e., full of zeros), and adjust the number of networks as well as the parameters scalings.
### Of course the Forward function will also check the self.IsParamRedundant attribute to see which Parameters
### to use.
        print('====== Checking parameter redundancy. ======')
        
        Param_idx = torch.arange(Parameters.size(1))
        zero_idx  = (torch.nonzero(Parameters[0] == Parameters[1]+Parameters[0])) # possible zero columns
        zero_mask = torch.tensor([len(torch.nonzero(Parameters[:, zero_idx.squeeze()] !=0)
                                     )!=0 if idx in zero_idx else True for idx in Param_idx])
        self.IsParamRedundant = (sum(zero_mask) != Parameters.size(1))
        print('====== IsParamRedundant: ' + str(self.IsParamRedundant))
        if self.IsParamRedundant:
            good_parameters = torch.nonzero(zero_mask)
            print(good_parameters)
            print('====== Effective parameters: ' + str(list(good_parameters)))
            self.Parameters_Short = Parameters[:, good_parameters]
            if Parameters.is_cuda():
                self.Parameters_Short = self.Parameters_Short.cuda()
            self.DefineLayers(len(good_parameters))
            

    def InitPreprocess(self, Data, Parameters):
### This can be run only ONCE to initialize the preprocess (shift and scaling) parameters
### Takes as input the training Data and the training Parameters as Torch tensors.
        
        # check redunancy
        self.checkRedundancy(Parameters)
        
        if not hasattr(self, 'Scaling'):
            print('Initializing Preprocesses Variables')
            self.Scaling = Data.std(0)
            self.Shift = Data.mean(0)
            self.ParameterScaling = self.Parameters_Short.std(0
                                ) if self.IsParamRedundant else Parameters.std(0)             
        else: print('Preprocess can be initialized only once. Parameters unchanged.')
            
    def Preprocess(self, Data, Parameters):
### Returns scaled/shifted data and parameters
### Takes as input Data and Parameters as Torch tensors.
        if  not hasattr(self, 'Scaling'): print('Preprocess parameters are not initialized.')
        Data = (Data - self.Shift)/self.Scaling
        Parameters = Parameters/self.ParameterScaling
        Ones = torch.ones([Parameters.size(0),1], dtype=Parameters.dtype)
        if Parameters.is_cuda:
            Ones = Ones.cuda()
        Parameters = torch.cat([Ones, Parameters.reshape(Data.size(0), -1)], dim=1)
        return Data, Parameters
    
    def Save(self, Name, Folder, csvFormat=False):
### Saves the model in Folder/Name
        FileName = Folder + Name + '.pth'
        torch.save({'StateDict': self.state_dict(), 
                   'Scaling': self.Scaling,
                   'Shift': self.Shift,
                   'ParameterScaling': self.ParameterScaling}, 
                   FileName)
        print('Model successfully saved.')
        print('Path: %s'%str(FileName))
        
        if csvFormat:
            modelparams = [w.detach().tolist() for w in self.parameters()]
            np.savetxt(Folder + Name + ' (StateDict).csv', modelparams, '%s')
            statistics = [self.Shift.detach().tolist(), self.Scaling.detach().tolist(),
                         self.ParameterScaling.detach().tolist()]
            np.savetxt(Folder + Name + ' (Statistics).csv', statistics, '%s')
    
    def Load(self, Name, Folder):
### Loads the model from Folder/Name
        FileName = Folder + Name + '.pth'
        try:
            IncompatibleKeys = self.load_state_dict(torch.load(FileName)['StateDict'])
        except KeyError:
            print('No state dictionary saved. Loading model failed.')
            return 
        
        if list(IncompatibleKeys)[0]:
            print('Missing Keys: %s'%str(list(IncompatibleKeys)[0]))
            print('Loading model failed. ')
            return 
        
        if list(IncompatibleKeys)[1]:
            print('Unexpected Keys: %s'%str(list(IncompatibleKeys)[0]))
            print('Loading model failed. ')
            return 
        
        self.Scaling = torch.load(FileName)['Scaling']
        self.Shift = torch.load(FileName)['Shift']
        self.ParameterScaling = torch.load(FileName)['ParameterScaling']
        
        print('Model successfully loaded.')
        print('Path: %s'%str(FileName))
        
    def Report(self): ### is it possibe to check if the model is in double?
        print('\nModel Report:')
        print('Preprocess Initialized: ' + str(hasattr(self, 'Shift')))
        print('Architecture: ' + str(self.Architecture))
        print('Loss Function: ' + 'Quadratic')
        print('Activation: ' + str(self.ActivationFunction))
        
    def cuda(self):
        nn.Module.cuda(self)
        self.Shift = self.Shift.cuda()
        self.Scaling = self.Scaling.cuda()
        self.ParameterScaling = self.ParameterScaling.cuda()
        
    def cpu(self):
        self.Shift = self.Shift.cpu()
        self.Scaling = self.Scaling.cpu()
        self.ParameterScaling = self.ParameterScaling.cpu()
        return nn.Module.cpu(self)



In [130]:
DataFolder = '/data3/WZ_new_project/h5/Ideal_Data'

td = OurTrainingData([DataFolder + '/ChP_pt300_sm_1.h5',
                     ],
                     [DataFolder + '/ChP_pt300_gphim5e-1.h5',
                     DataFolder + '/ChP_pt300_gphim2e-1.h5',
                     DataFolder + '/ChP_pt300_gphim5e-2.h5',
                     DataFolder + '/ChP_pt300_gphi5e-1.h5',
                     DataFolder + '/ChP_pt300_gphi2e-1.h5',
                     DataFolder + '/ChP_pt300_gphi5e-2.h5'],
                     process = 'W+Z', parameters =['Gphi[TeV**-2]', 'GW[TeV**-2]'], 
                     SMNLimits=int(3e6),
                     BSMNLimits=int(5e5))

NumEpochs = int(1e4)

td.Data = td.Data[:, :7]
td.CurateAngles([3, 5])

Data, ParVal, Labels, Weights = td.Data, td.ParVal, td.Labels, td.Weights
Data, ParVal, Labels, Weights = Data.float(), ParVal.float(), Labels.float(), Weights.float()

Loading Data Files for Process: W+Z, with new physics Parameters: ['Gphi[TeV**-2]', 'GW[TeV**-2]']

Reading file .../data3/WZ_new_project/h5/Ideal_Data/ChP_pt300_gphim5e-1.h5
##### File Info:
{Gphi[TeV**-2], GW[TeV**-2]} = {-0.5, 0.}[TeV**-2] data, Ideal Events. 
Event format: {{s, θ, θZ, ϕZ, θWrec, ϕWrec, PtZ}, weight}.
Converted from /data3/WZ_new_project/dat/Ideal_Events/ChP_pt300_gphim5e-1.dat.gz
Charge = 1 --- Process = W+Z
#####

Reading file .../data3/WZ_new_project/h5/Ideal_Data/ChP_pt300_gphim2e-1.h5
##### File Info:
{Gphi[TeV**-2], GW[TeV**-2]} = {-0.2, 0.}[TeV**-2] data, Ideal Events. 
Event format: {{s, θ, θZ, ϕZ, θWrec, ϕWrec, PtZ}, weight}.
Converted from /data3/WZ_new_project/dat/Ideal_Events/ChP_pt300_gphim2e-1.dat.gz
Charge = 1 --- Process = W+Z
#####

Reading file .../data3/WZ_new_project/h5/Ideal_Data/ChP_pt300_gphim5e-2.h5
##### File Info:
{Gphi[TeV**-2], GW[TeV**-2]} = {-0.05, 0.}[TeV**-2] data, Ideal Events. 
Event format: {{s, θ, θZ, ϕZ, θWrec, ϕWrec, PtZ}, weigh

In [47]:
mute_idx = (torch.nonzero(ParVal[0] == ParVal[1]+ParVal[0]))

mute_idx = (torch.nonzero(ParVal[0] == ParVal[1]))

mute_idx

tensor([[0],
        [1]])

In [51]:
#mute_candidate = torch.tensor([[1,],[0,]])
#mute_candidate = torch.tensor([[1,]])

for col_idx in mute_idx:
    print(len(torch.nonzero(ParVal[:, col_idx]!=0))==0) 

False
True


In [65]:
zero_idx = (torch.nonzero(ParVal[0] == ParVal[1]+ParVal[0])) # possible zero columns
zero_idx = torch.tensor([len(torch.nonzero(ParVal[:, col_idx]==0))==0 for col_idx in zero_idx]) # check for zeros
ParVal_Short = ParVal[:, torch.nonzero(zero_idx).squeeze()]
ParVal

tensor([[-0.5000,  0.0000],
        [-0.5000,  0.0000],
        [-0.5000,  0.0000],
        ...,
        [ 0.0500,  0.0000],
        [ 0.0500,  0.0000],
        [ 0.0500,  0.0000]])

In [70]:
zero_idx = (torch.nonzero(ParVal[0] == ParVal[1]+ParVal[0])) # possible zero columns
#zero_idx = torch.tensor([len(torch.nonzero(ParVal[:, col_idx]==0))==0 for col_idx in zero_idx]) # check for zeros
[(torch.nonzero(ParVal[:, col_idx]==0)) for col_idx in zero_idx]
#zero_idx

[tensor([[      0,       0],
         [      1,       0],
         [      2,       0],
         ...,
         [5999997,       0],
         [5999998,       0],
         [5999999,       0]])]

In [107]:
zero_idx = (torch.nonzero(ParVal[0] == ParVal[1]+ParVal[0])) # possible zero columns
zero_mask = torch.tensor([len(torch.nonzero(ParVal[:, zero_idx.squeeze()] !=0)
                             )!=0 if idx in zero_idx else True for idx in ParVal_idx])
ParVal_Short = ParVal[:, torch.nonzero(zero_mask).squeeze()]
sum(zero_mask) == ParVal.size(1)

In [109]:
ParVal_Short = ParVal[:, torch.nonzero(zero_mask).squeeze()]

In [120]:
torch.nonzero(zero_mask).squeeze()

tensor(0)

In [118]:
torch.arange(ParVal.size(1))

tensor([0, 1])